## 2e. Calculate closed-system degassing paths for different volatile systems.

## Setting things up
First we need to import a few Python packages (including VolFe) and set the precision of the calculations. You need to install VolFe once on your machine, if you haven't yet, uncomment the line below (remove the #).

In [ ]:
# Install VolFe on your machine. Remove the # from line below to do this (don't remove the # from this line!).
# pip install VolFe

In [1]:
# import python packages
import pandas as pd
import gmpy2 as gp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import VolFe as vf

# set calculation precision
gp.get_context().precision=50

These are some examples where the volatiles aren't C, H, and S. These types of calculations can be done for open- and closed-system, re- and degassing paths as described in Examples 2a-d, but we'll just show them for closed-system degassing paths where the inputted composition represents the bulk composition of the system.

For the volatile-free melt composition, we'll use Sari15-04-33 from Brounce et al. (2014) at 1200 'C as before, but we will specify different volatile concentrations. 
First, we'll model CHOAr degassing - i.e., carbon, hydrogen, and argon are the volatiles of interest (no sulfur - at the moment it isn't possible to run CHOS and a noble gas).
In this case the initial volatile content is 500 ppm CO2, 2 wt%, and 100 ppm Ar.
The Ar amount is inputted for the "X" species.

In [8]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Sari15-04-33',
           'T_C': 1200., # Temperature in 'C
           'SiO2': 47.89, # wt%
           'TiO2': 0.75, # wt%
           'Al2O3': 16.74, # wt%
           'FeOT': 9.43, # wt%
           'MnO': 0.18, # wt%
           'MgO': 5.92, # wt%
           'CaO': 11.58, # wt%
           'Na2O': 2.14, # wt%
           'K2O': 0.63, # wt%
           'P2O5': 0.17, # wt%
           'H2O': 2., # wt%
           'CO2ppm': 500., # ppm
           'STppm': 0., # ppm
           'Xppm': 10., # ppm <<< treating this as Ar
           'Fe3FeT': 0.195}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

We'll use the default options, but the important options for noble gas modelling are:

**species X** where the default is Ar (*Ar*)

**species X solubility** where the default is for Ar in basalt (*Ar_Basalt_HughesIP*)

Both these defaults are fine for this example, but you can change them to Ar in rhyolite (*Ar_Rhyolite_HughesIP*) or Ne (*Ne*) for basalt (*Ne_Basalt_HughesIP*) or rhyolite (*Ne_Rhyolite_HughesIP*).

In [ ]:
# print default options in VolFe
print(vf.default_models)

In [9]:
# choose the options I want - everything else will use the default options
my_models = [['print status','yes']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

And below runs the calculation

In [10]:
degas1 = vf.calc_gassing(my_analysis, models=my_models)

Sari15-04-33
1200.0 1460.1133014759525 2024-04-07 09:01:09.532771
solver failed, calculation aborted at P =  1450.0 2024-04-07 09:01:26.282764


In [ ]:
# choose the options I want - everything else will use the default options
my_models = [['sulfur_saturation','yes']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

And then we run the degassing calculation with those model options

In [ ]:
degas2 = vf.calc_gassing(my_analysis, models=my_models)

In [ ]:
print(degas1["CO2T-eq_ppmw"])

And below we can plot the two results to see the difference.
The blue curves are the first calculation (not allowing sulfur to saturate), whilst the red curves are the second calculation (sulfur can saturate as sulfide or anhydrite if the concentration in the melt is high enough).
It has no effect on the H2O or CO2 contents of the melt, but the sulfur content is much less at the start of degassing because it is in a sulfide phase instead!

In [ ]:
fig = make_subplots(rows=1, cols=3, shared_yaxes = True, shared_xaxes = False,vertical_spacing=0.08, horizontal_spacing=0.05)

lw = 3
syms = 10
lc1 = "steelblue"
lc2 = "firebrick"

data1 = degas1
data2 = degas2

r = 1
# P vs. CO2
c = 1
fig.add_trace(go.Scatter(mode = "lines", x=data1['CO2T-eq_ppmw'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
#fig.add_trace(go.Scatter(mode = "lines", x=data2['CO2T-eq_ppmw'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "CO<sub>2</sub>-eq (ppm)", row = r, col = c)
fig.update_yaxes(title = "<i>P</i> (bars)", row = r, col = c)

# P vs. H2O
c = 2
#fig.add_trace(go.Scatter(mode = "lines", x=data1['H2OT-eq_wtpc'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
#fig.add_trace(go.Scatter(mode = "lines", x=data2['H2OT-eq_wtpc'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "H<sub>2</sub>O-eq (wt%)", row = r, col = c)

# P vs. S
c = 3
#fig.add_trace(go.Scatter(mode = "lines", x=data1['ST_ppmw'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
#fig.add_trace(go.Scatter(mode = "lines", x=data2['ST_ppmw'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "S<sub>T</sub> (ppm)", row = r, col = c)

c = 1
#fig.add_trace(go.Scatter(mode = "lines", x=data1['fO2_DFMQ'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
#fig.add_trace(go.Scatter(mode = "lines", x=data2['fO2_DFMQ'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "DFMQ", row = r, col = c)

fig.update_layout(height=400, width=1000, plot_bgcolor='rgb(255,255,255)' , showlegend = False)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15, family='Helvetica', color='black'), tickfont=dict(family='Helvetica', color='black', size=12))
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15, family='Helvetica', color='black'), tickfont=dict(family='Helvetica', color='black', size=12))

fig.update_layout(font_family="Helvetica",font_color="black")

fig.show()